In [1]:
!git clone https://github.com/pranit570/A2Bnotoll1nov.git

Cloning into 'A2Bnotoll1nov'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [14]:
!ls A2Bnotoll1nov


a2bnotoll2.mp4	modifiedarchjan20.h5  rawimagefromvdo  README.md  trimmedvdo


In [15]:
from keras.layers import Input, Dense, merge
from keras.models import Model
from keras.layers import MaxPooling2D, Reshape, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam

In [16]:
def default_categorical():
  img_in = Input(shape=(66, 200, 3), name='img_in')
  x = img_in
  x = Conv2D(26, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu', name='conv1')(x)
  x = Conv2D(38, (5, 5), strides=(2, 2), activation='elu', name='conv2')(x)
  x = Conv2D(52, (5, 5), strides=(2, 2), activation='elu', name='conv3')(x)
  x = Conv2D(64, (3, 3), activation='elu', name='conv4')(x)  
  x = Conv2D(128, (3, 3), activation='elu',name='conv5')(x)
  #x = Dropout(0.5)(x)  
  x = Flatten(name='flattened')(x)  
  x = Dense(100, activation = 'elu', name = 'dense1')(x)
  #x = Dropout(0.5)(x)  
  x = Dense(50, activation = 'elu', name = 'dense2')(x)
  #x = Dropout(0.5)(x)  

  angle_out = Dense(10, activation = 'elu', name = 'angle_out')(x)
  x = angle_out 

  throttle_out = Dense(1, activation = 'elu', name = 'throttle_out')(x)

  model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])
  optimizer = Adam(lr=1e-3)
  model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
  return model

In [17]:
model=default_categorical()
model.load_weights('/content/A2Bnotoll1nov/modifiedarchjan20.h5')


In [18]:
img_in = Input(shape=(66, 200, 3), name='img_in')
x = img_in
x = Conv2D(26, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu', name='conv1')(x)
x = Conv2D(38, (5, 5), strides=(2, 2), activation='elu', name='conv2')(x)
x = Conv2D(52, (5, 5), strides=(2, 2), activation='elu', name='conv3')(x)
x = Conv2D(64, (3, 3), activation='elu', name='conv4')(x)  
conv_5 = Conv2D(128, (3, 3), activation='elu',name='conv5')(x)
convolution_part = Model(inputs=[img_in], outputs=[conv_5])

In [19]:
for layer_num in ('1', '2', '3', '4', '5'):
    convolution_part.get_layer('conv' + layer_num).set_weights(model.get_layer('conv' + layer_num).get_weights())

In [20]:
from keras import backend as K

inp = convolution_part.input                                           # input placeholder
outputs = [layer.output for layer in convolution_part.layers[1:]]          # all layer outputs
functor = K.function([inp], outputs)

In [21]:
import tensorflow as tf

In [22]:
import numpy as np
import pdb

In [23]:
kernel_3x3 = tf.constant(np.array([
        [[[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]]]
]), tf.float32)

kernel_5x5 = tf.constant(np.array([
        [[[1]], [[1]], [[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]], [[1]], [[1]]],
        [[[1]], [[1]], [[1]], [[1]], [[1]]],
        [[[1]], [[1]], [[1]], [[1]], [[1]]]
]), tf.float32)

layers_kernels = {5: kernel_3x3, 4: kernel_3x3, 3: kernel_5x5, 2: kernel_5x5, 1: kernel_5x5}

layers_strides = {5: [1,1,1, 1], 4: [1,1,1, 1], 3: [1, 2, 2, 1], 2: [1, 2, 2, 1], 1: [1, 2, 2, 1]}

def compute_visualisation_mask(img):
#     pdb.set_trace()
    activations = functor([np.array([img])])
    activations = [np.reshape(img, (1, img.shape[0], img.shape[1], img.shape[2]))] + activations
    upscaled_activation = np.ones((1, 18))
    for layer in [5, 4, 3, 2, 1]:
        averaged_activation = np.mean(activations[layer], axis=3).squeeze(axis=0) * upscaled_activation
        output_shape = (activations[layer - 1].shape[1], activations[layer - 1].shape[2])
        x = tf.constant(
            np.reshape(averaged_activation, (1,averaged_activation.shape[0],averaged_activation.shape[1],1)),
            tf.float32
        )
        conv = tf.nn.conv2d_transpose(
            x, layers_kernels[layer],
            output_shape=(1,output_shape[0],output_shape[1], 1), 
            strides=layers_strides[layer], 
            padding='VALID'
        )
      #  with tf.compat.v1.Session() as session:
       #     result = session.run(conv)
        upscaled_activation = np.reshape(conv, output_shape)
    final_visualisation_mask = upscaled_activation
    return (final_visualisation_mask - np.min(final_visualisation_mask))/(np.max(final_visualisation_mask) - np.min(final_visualisation_mask))

In [24]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [25]:
from glob import iglob

In [92]:
imgs = []
alpha = 0.006
beta = 1.0 - alpha
counter = 0
for path in sorted(iglob('/content/A2Bnotoll1nov/rawimagefromvdo/*.jpg')):
    
    img = cv2.imread(path)
    img = img[700:925,530:1255,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    #img = img/255
    salient_mask = compute_visualisation_mask(img)
    salient_mask_stacked = np.dstack((salient_mask,salient_mask))
    salient_mask_stacked = np.dstack((salient_mask_stacked,salient_mask))
    blend = cv2.addWeighted(img.astype('float32'), alpha, salient_mask_stacked, beta, 0.0)
    imgs.append(blend)
    counter += 1
    if counter >= 1000:
        break


In [93]:
print(len(imgs))

553


In [94]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML

def plot_movie_mp4(image_array):
    dpi = 600.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)   

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    display(HTML(anim.to_html5_video()))

In [95]:
plot_movie_mp4(imgs)


<Figure size 200x66 with 0 Axes>

In [ ]:
imname=""
for i in range(len(imgs)):
  if i<10: 
      imname= "00"+str(i)
  if 10<=i<=99:
      imname="0"+str(i)
  if i>99:
      imname= str(i) 
  
  cv2.imwrite('/content/a2btoll/output2/'+imname+'.jpg',imgs[i])
           

In [ ]:
import cv2
import os

# Opens the Video file
cap= cv2.VideoCapture('/content/a2btoll/A2Btoll.mp4')

i=0
fname=""
while(cap.isOpened()):
    if i<10: 
      fname= "00"+str(i)
    if 10<=i<=99:
      fname="0"+str(i)
    if i>99:
      fname= str(i) 
    ret, frame = cap.read()
    if ret == False:
        break
   # cv2.imwrite('/content/a2btoll/test2/'+str(i)+'.jpg',frame)
    cv2.imwrite('/content/a2btoll/test3/'+fname+'.jpg',frame)
    i+=1
    


In [ ]:
!zip -r /content/salientrightimages/salientimages.zip /content/salientrightimages/salientimages

In [ ]:
import cv2
import os

# Opens the Video file
cap= cv2.VideoCapture('/content/salientrightimages/download.mp4')

i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('/content/salientrightimages/downloadimg/download'+str(i)+'.jpg',frame)
    i+=1



In [ ]:
!zip -r /content/salientrightimages/downloadimg.zip /content/salientrightimages/downloadimg

In [ ]:
import cv2

# Opens the Video file
cap= cv2.VideoCapture('/content/salientrightimages/salientallobstacle.mp4')

i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('/content/salientrightimages/salientallobstacleimg/salientimg'+str(i)+'.jpg',frame)
    i+=1



In [ ]:
!zip -r /content/salientrightimages/salientallobstacleimg.zip /content/salientrightimages/salientallobstacleimg

In [5]:
#getting the raw video with trimmed image sizes
import cv2
import os

# Opens the Video file
cap= cv2.VideoCapture('/content/A2Bnotoll1nov/a2bnotoll2.mp4')

i=0
fname=""
while(cap.isOpened()):
    if i<10: 
      fname= "00"+str(i)
    if 10<=i<=99:
      fname="0"+str(i)
    if i>99:
      fname= str(i) 
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('/content/A2Bnotoll1nov/rawimagefromvdo/'+fname+'.jpg',frame)
    i+=1

In [10]:
#trimming and resizing images
from glob import iglob
imgs = []
alpha = 0.003
beta = 1.0 - alpha
counter = 0
for path in sorted(iglob('/content/A2Bnotoll1nov/rawimagefromvdo/*.jpg')):
    
    img = cv2.imread(path)
    img = img[700:925,530:1255,:]
    #img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    #img = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    #img = img/255
    #salient_mask = compute_visualisation_mask(img)
    #salient_mask_stacked = np.dstack((salient_mask,salient_mask))
    #salient_mask_stacked = np.dstack((salient_mask_stacked,salient_mask))
    #blend = cv2.addWeighted(img.astype('float32'), alpha, salient_mask_stacked, beta, 0.0)
    imgs.append(img)
    counter += 1
    if counter >= 1000:
        break

In [11]:
print(len(imgs))

553


In [12]:
#generate a vdo
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML

def plot_movie_mp4(image_array):
    dpi = 600.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)   

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    display(HTML(anim.to_html5_video()))

In [13]:
plot_movie_mp4(imgs)

<Figure size 200x66 with 0 Axes>